In [ ]:
!pip install node2vec
!pip install --upgrade gensim
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

  Created wheel for node2vec: filename=node2vec-0.4.3-py3-none-any.whl size=5980 sha256=8525d5bc29839dc44242d429433fef0d6822328e12ba70b1a1a5858b1e111879
  Stored in directory: /root/.cache/pip/wheels/07/62/78/5202cb8c03cbf1593b48a8a442fca8ceec2a8c80e22318bae9
Successfully built node2vec
     |████████████████████████████████| 24.1 MB 1.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 7.9 MB 8.7 MB/s 
     |████████████████████████████████| 3.5 MB 9.8 MB/s 


In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import pandas as pd
import numpy as np 
import random
random.seed(2022)
np.random.seed(2022)

In [ ]:
!pip install causalml

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv
Saving synthetic_edge_index.csv to synthetic_edge_index.csv


In [ ]:
uploaded2 = files.upload()

Saving synthetic_deepwalk_10d.model to synthetic_deepwalk_10d.model
Saving synthetic_Node2Vec_10d.model to synthetic_Node2Vec_10d.model


In [ ]:
uploaded3 = files.upload()

Saving Synthetic_feats_ate.xlsx to Synthetic_feats_ate.xlsx


In [ ]:
import io
df=pd.read_csv(io.BytesIO(uploaded['data.csv']),index_col=0)

from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df,test_size = 0.2,random_state=42)
df

,X0,X1,X2,X3,X4,T,y,tau,b,e
0,0.003154,0.402130,0.582037,0.539144,0.414101,0,0.743337,0.202642,0.763640,0.100000
1,0.594773,0.034123,0.250783,0.094229,0.148447,0,0.598364,0.314448,0.356388,0.100000
2,0.403102,0.952464,0.887029,0.141932,0.376224,1,3.017079,0.677783,1.563889,0.900000
3,0.177169,0.367340,0.704904,0.043881,0.904914,0,0.342602,0.272255,0.783347,0.203037
4,0.492973,0.030507,0.002244,0.535752,0.489047,0,-0.501458,0.261740,1.323027,0.100000
...,...,...,...,...,...,...,...,...,...,...
9995,0.989169,0.379821,0.512016,0.444698,0.915355,1,3.888099,0.684495,1.827391,0.900000
9996,0.604783,0.351680,0.938821,0.560349,0.205232,1,1.879708,0.478232,1.667655,0.619563
9997,0.300281,0.457595,0.810301,0.170581,0.925457,0,0.616104,0.378938,1.244277,0.418394
9998,0.513388,0.990300,0.266999,0.770421,0.307555,1,2.053641,0.751844,2.032429,0.900000


In [ ]:
col=df.columns
y_train=torch.from_numpy(np.array((df_train['y']))).reshape(df_train.shape[0],1).to(torch.float32)
y_test=torch.from_numpy(np.array(df_test['y'])).reshape(df_test.shape[0],1).to(torch.float32)
x_train=torch.from_numpy(np.array(df_train[col[:6]])).to(torch.float32)
x_test=torch.from_numpy(np.array(df_test[col[:6]])).to(torch.float32)

train_reverse=df_train[col[:6]].copy()
train_reverse["T"]=1-train_reverse["T"]
train_reverse=torch.from_numpy(np.array(train_reverse)).to(torch.float32)
test_reverse=df_test[col[:6]].copy()
test_reverse["T"]=1-test_reverse["T"]
test_reverse=torch.from_numpy(np.array(test_reverse)).to(torch.float32)

In [ ]:
from scipy.stats import entropy
def R2(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    R2 = 1 - np.sum(np.square(y_actual-y_predicted)) / np.sum(np.square(y_actual-np.mean(y_actual)))
    return R2

def MAE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAE = np.mean(abs(y_actual-y_predicted))
    return MAE

def RMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    return RMSE

def CVRMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    CVRMSE = RMSE/np.mean(y_actual)
    return CVRMSE

def MAPE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAPE=np.mean(abs((y_actual-y_predicted)/y_actual))
    return MAPE

def MSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MSE = np.mean(abs(y_actual-y_predicted)**2)
    return MSE

def kl_divergence(p, q):

    p = np.asarray(p, dtype=float)
    q = np.asarray(q, dtype=float)
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

def kl_divergence(y_predicted, y_actual):

    stacked_values = np.hstack((y_predicted, y_actual))
    stacked_low = np.percentile(stacked_values, 0.1)
    stacked_high = np.percentile(stacked_values, 99.9)
    bins = np.linspace(stacked_low, stacked_high, 100)

    distr = np.histogram(y_predicted, bins=bins)[0]
    distr = np.clip(distr / distr.sum(), 0.001, 0.999)
    true_distr = np.histogram( y_actual, bins=bins)[0]
    true_distr = np.clip(true_distr / true_distr.sum(), 0.001, 0.999)

    kl = entropy(distr, true_distr)
    return kl 


In [ ]:
edge_index=pd.read_csv(io.BytesIO(uploaded['synthetic_edge_index.csv']))
edge_index=torch.from_numpy(np.transpose(np.array(edge_index)))

In [ ]:
from gensim.models import word2vec
model_dw=word2vec.Word2Vec.load("synthetic_deepwalk_10d.model")
model_n2v=word2vec.Word2Vec.load("synthetic_Node2Vec_10d.model")

lst_dw=[]
lst_n2v=[]
for i in range(6):
    lst_dw.append(model_dw.wv[i])
    lst_n2v.append(model_n2v.wv[i])
len(lst_n2v)

6

In [ ]:
ate_list=[]
ate=pd.read_excel(io.BytesIO(uploaded3["Synthetic_feats_ate.xlsx"]),index_col=0)
for i in col[:6]:
    ate_list.append(float(ate[ate['Feature']==i]["ATE"].values))
    
ate_list

[0.6138629231035895,
 0.6256793856791922,
 0.001699585840328169,
 0.9901459828003378,
 0.4672027205639851,
 0.5306717299934571]

In [ ]:
import torch
from torch_geometric.data import InMemoryDataset, download_url

# create Mytrain Data
class Mytrain(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(x_train.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =x_train[i]
            t=torch.zeros(6,17) # 17 dimensions = 1+10+6 = node number, node embedding, weight*feature

            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(causal_weighted)
            
            for j in range(6): # for j-th feature
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec
                                                     list(causal_weighted) # weight*feature (6d), causal-weighted feature        
                                                    ))) 
              
            Y = y_train[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# create Mytest Data
class Mytest(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        
   
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(x_test.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =x_test[i]
            t=torch.zeros(6,17) # 17 dimensions = 1+10+6 = node number, node embedding, weight*feature

            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(causal_weighted)
            
            for j in range(6): # for j-th feature
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec
                                                     list(causal_weighted) # weight*feature (6d), causal-weighted feature        
                                                    ))) 

            Y = y_test[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# create Mytrain_reverse Data
class Mytrain_reverse(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
    
    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(train_reverse.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =train_reverse[i]
            t=torch.zeros(6,17) # 17 dimensions = 1+10+6 = node number, node embedding, weight*feature
            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(causal_weighted)
            
            for j in range(6): # for j-th feature
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec
                                                     list(causal_weighted) # weight*feature (6d), causal-weighted feature        
                                                    ))) 


            Y = y_train[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])


# create Mytest_reverse Data
class Mytest_reverse(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(test_reverse.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =test_reverse[i]
            t=torch.zeros(6,17) # 17 dimensions = 1+10+6 = node number, node embedding, weight*feature
            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(causal_weighted)
            
            for j in range(6): # for j-th feature
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec
                                                     list(causal_weighted) # weight*feature (6d), causal-weighted feature        
                                                    ))) 


            Y = y_test[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)

            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
Mydata_train=Mytrain(".\mydata\MYdata_train")
Mydata_test=Mytest(".\mydata\MYdata_test")
Mytrain_reverse=Mytrain_reverse(".\mydata\MYtrain_reverse")
Mytest_reverse=Mytest_reverse(".\mydata\MYtest_reverse")

Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!


# GCN

In [ ]:
import torch
from torch_scatter import scatter_add
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(17, 64)
        self.conv2 = GCNConv(64, 21)
        self.f1 = torch.nn.Linear(126,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = x.reshape(-1,126)
        x = self.f1(x)
        x = self.f2(x)
        return x

In [ ]:
def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in train_loader:
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(Mydata_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in test_loader:
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(Mydata_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl

    

te_train = {}
te_valid = {}
for i in range(5):
    train_mse=[]
    train_kl=[]
    train_error=[]

    valid_mse=[]
    valid_kl=[]
    valid_error=[]

    num_epochs = 2560
    batch_size = 512
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
    crit = F.mse_loss
    train_loader = DataLoader(Mydata_train, batch_size=batch_size)
    test_loader = DataLoader(Mydata_test, batch_size=512)
    reverse_train = DataLoader(Mytrain_reverse, batch_size=512)
    reverse_test = DataLoader(Mytest_reverse, batch_size=512)
    for epoch in range(num_epochs):
        loss,r2,mse,kl=train()
        if epoch %64==0:      
            print('train_loss:')
            print(loss)

            loss,r2,mse,kl=val()
            print('test_loss:')
            print(loss)
  
    y_predicted = []
    for data in train_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_train["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_train:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_train["y_reversed"]=y_reversed
    t0=df_train[df_train["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_train[df_train["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    train_mse.append(mse)
    train_kl.append(kl)
    train_error.append(ate_error)

    print("Model %d has MSE on trainset:%f" % (i,np.mean(train_mse)),end='  ')
    print("Model %d has KL on trainset:%f" % (i,np.mean(train_kl)),end='  ')
    print("Model %d has ATE error on trainset:%f" % (i,np.mean(train_error)),end='  ')

    te_train["GCN ATE % Error",i]=np.mean(train_error)
    te_train["GCN MSE",i]=np.mean(train_mse)
    te_train["GCN KL Divergence",i]=np.mean(train_kl)

    y_predicted = []
    for data in test_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_test["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_test:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_test["y_reversed"]=y_reversed
    t0=df_test[df_test["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_test[df_test["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    valid_mse.append(mse)
    valid_kl.append(kl)
    valid_error.append(ate_error)

    print("Model %d has MSE on validset:%f" % (i,np.mean(valid_mse)),end='  ')
    print("Model %d has KL on validset:%f" % (i,np.mean(valid_kl)),end='  ')
    print("Model %d has ATE error on validset:%f" % (i,np.mean(valid_error)),end='  ')

    te_valid["GCN ATE % Error",i]=np.mean(valid_error)
    te_valid["GCN MSE",i]=np.mean(valid_mse)
    te_valid["GCN KL Divergence",i]=np.mean(valid_kl)
    print(te_valid)

R2:-0.835154  MSE:2.556351  KL:0.960546  MAE:1.299759  RMSE:1.598859  CVRMSE:1.055962  train_loss:
2.556351221084595
R2:0.132035  MSE:1.268306  KL:0.701477  MAE:0.897008  RMSE:1.126191  CVRMSE:0.743576  test_loss:
0.002534283757209778
R2:0.088162  MSE:1.270181  KL:0.405077  MAE:0.899771  RMSE:1.127023  CVRMSE:0.744339  R2:0.178932  MSE:1.143739  KL:0.397411  MAE:0.853649  RMSE:1.069457  CVRMSE:0.706320  R2:0.214726  MSE:1.093878  KL:0.267432  MAE:0.833151  RMSE:1.045886  CVRMSE:0.690753  R2:0.222890  MSE:1.082507  KL:0.250852  MAE:0.827418  RMSE:1.040436  CVRMSE:0.687153  R2:0.223142  MSE:1.082156  KL:0.252468  MAE:0.827547  RMSE:1.040267  CVRMSE:0.687042  R2:0.226855  MSE:1.076983  KL:0.242229  MAE:0.826694  RMSE:1.037778  CVRMSE:0.685398  R2:0.230918  MSE:1.071323  KL:0.240245  MAE:0.823604  RMSE:1.035047  CVRMSE:0.683594  R2:0.238592  MSE:1.060633  KL:0.239797  MAE:0.818778  RMSE:1.029871  CVRMSE:0.680175  R2:0.239829  MSE:1.058910  KL:0.236655  MAE:0.818717  RMSE:1.029033  CVRMSE:0

In [ ]:
te_train,te_valid

({('GCN ATE % Error', 0): 0.024331848706911074,
  ('GCN ATE % Error', 1): 0.04138872832019602,
  ('GCN ATE % Error', 2): 0.07992515441508585,
  ('GCN ATE % Error', 3): 0.010626119847234827,
  ('GCN ATE % Error', 4): 0.012687682422655833,
  ('GCN KL Divergence', 0): 0.30315243805992076,
  ('GCN KL Divergence', 1): 0.29713713910991735,
  ('GCN KL Divergence', 2): 0.4079997372251049,
  ('GCN KL Divergence', 3): 0.33518217753422336,
  ('GCN KL Divergence', 4): 0.39222105612336344,
  ('GCN MSE', 0): 0.12288437669255284,
  ('GCN MSE', 1): 0.13207905684870902,
  ('GCN MSE', 2): 0.15126790124809428,
  ('GCN MSE', 3): 0.1206743974331404,
  ('GCN MSE', 4): 0.14288008126519203},
 {('GCN ATE % Error', 0): 0.04021738046212642,
  ('GCN ATE % Error', 1): 0.017221483589586195,
  ('GCN ATE % Error', 2): 0.10788412166935646,
  ('GCN ATE % Error', 3): 0.012653099666505074,
  ('GCN ATE % Error', 4): 0.01127150097245877,
  ('GCN KL Divergence', 0): 0.32190726215661036,
  ('GCN KL Divergence', 1): 0.3516931

# GAT

In [ ]:
from uuid import RFC_4122
import torch
import math
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GATConv

from torch_geometric.utils import add_self_loops,degree
from torch_geometric.datasets import Planetoid
import ssl
import torch.nn.functional as F


class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.gat1=GATConv(in_channels=17,out_channels=8,heads=8,dropout=0.6)
        self.gat2=GATConv(in_channels=64,out_channels=21,heads=1,dropout=0.6)
        self.f1 = torch.nn.Linear(126,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self,data):
        x,edge_index=data.x, data.edge_index
        x=self.gat1(x,edge_index)
        x=self.gat2(x,edge_index)
        x=x.reshape(-1,126)
        x = self.f1(x)
        x = self.f2(x)
        return x

ssl._create_default_https_context = ssl._create_unverified_context
def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in train_loader:
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(Mydata_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in test_loader:
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(Mydata_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl

    
for i in range(5):
    train_mse=[]
    train_kl=[]
    train_error=[]

    valid_mse=[]
    valid_kl=[]
    valid_error=[]

    num_epochs = 2560
    batch_size = 512
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
    crit = F.mse_loss
    train_loader = DataLoader(Mydata_train, batch_size=batch_size)
    test_loader = DataLoader(Mydata_test, batch_size=512)
    reverse_train = DataLoader(Mytrain_reverse, batch_size=512)
    reverse_test = DataLoader(Mytest_reverse, batch_size=512)
    for epoch in range(num_epochs):
        loss,r2,mse,kl=train()
        if epoch %64==0:      
            print('train_loss:')
            print(loss)

            loss,r2,mse,kl=val()
            print('test_loss:')
            print(loss)
  
    y_predicted = []
    for data in train_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_train["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_train:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_train["y_reversed"]=y_reversed
    t0=df_train[df_train["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_train[df_train["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    train_mse.append(mse)
    train_kl.append(kl)
    train_error.append(ate_error)

    print("Model %d has MSE on trainset:%f" % (i,np.mean(train_mse)),end='  ')
    print("Model %d has KL on trainset:%f" % (i,np.mean(train_kl)),end='  ')
    print("Model %d has ATE error on trainset:%f" % (i,np.mean(train_error)),end='  ')

    te_train["GAT ATE % Error",i]=np.mean(train_error)
    te_train["GAT MSE",i]=np.mean(train_mse)
    te_train["GAT KL Divergence",i]=np.mean(train_kl)

    y_predicted = []
    for data in test_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_test["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_test:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_test["y_reversed"]=y_reversed
    t0=df_test[df_test["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_test[df_test["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    valid_mse.append(mse)
    valid_kl.append(kl)
    valid_error.append(ate_error)

    print("Model %d has MSE on validset:%f" % (i,np.mean(valid_mse)),end='  ')
    print("Model %d has KL on validset:%f" % (i,np.mean(valid_kl)),end='  ')
    print("Model %d has ATE error on validset:%f" % (i,np.mean(valid_error)),end='  ')

    te_valid["GAT ATE % Error",i]=np.mean(valid_error)
    te_valid["GAT MSE",i]=np.mean(valid_mse)
    te_valid["GAT KL Divergence",i]=np.mean(valid_kl)

R2:-0.252870  MSE:1.745235  KL:0.138346  MAE:1.051100  RMSE:1.321074  CVRMSE:0.872500  train_loss:
1.7452354135513306
R2:0.113662  MSE:1.295154  KL:0.645101  MAE:0.904412  RMSE:1.138048  CVRMSE:0.751405  test_loss:
0.0025913166403770446
R2:0.091721  MSE:1.265223  KL:0.255991  MAE:0.893963  RMSE:1.124821  CVRMSE:0.742885  R2:0.134951  MSE:1.205005  KL:0.277637  MAE:0.873512  RMSE:1.097727  CVRMSE:0.724991  R2:0.148236  MSE:1.186498  KL:0.256859  MAE:0.863871  RMSE:1.089265  CVRMSE:0.719402  R2:0.147895  MSE:1.186974  KL:0.259024  MAE:0.867715  RMSE:1.089483  CVRMSE:0.719546  R2:0.167875  MSE:1.159142  KL:0.264328  MAE:0.857783  RMSE:1.076634  CVRMSE:0.711060  R2:0.174661  MSE:1.149688  KL:0.265889  MAE:0.853886  RMSE:1.072235  CVRMSE:0.708155  R2:0.170357  MSE:1.155685  KL:0.261426  MAE:0.856566  RMSE:1.075028  CVRMSE:0.709999  R2:0.183511  MSE:1.137361  KL:0.266632  MAE:0.847202  RMSE:1.066471  CVRMSE:0.704348  R2:0.188842  MSE:1.129935  KL:0.273845  MAE:0.846014  RMSE:1.062984  CVRMSE

In [ ]:
te_train,te_valid

({('GAT ATE % Error', 0): 0.31780049158221657,
  ('GAT ATE % Error', 1): 0.3060487951856574,
  ('GAT ATE % Error', 2): 0.3306409568550175,
  ('GAT ATE % Error', 3): 0.31396108593053373,
  ('GAT ATE % Error', 4): 0.3214736167863348,
  ('GAT KL Divergence', 0): 0.3013906062883939,
  ('GAT KL Divergence', 1): 0.30320180816890674,
  ('GAT KL Divergence', 2): 0.3296618203655869,
  ('GAT KL Divergence', 3): 0.3114794888576304,
  ('GAT KL Divergence', 4): 0.3117788744111213,
  ('GAT MSE', 0): 0.09531270275320917,
  ('GAT MSE', 1): 0.09254813552964046,
  ('GAT MSE', 2): 0.09825510445250128,
  ('GAT MSE', 3): 0.09488353186055844,
  ('GAT MSE', 4): 0.09358193470273071,
  ('GCN ATE % Error', 0): 0.024331848706911074,
  ('GCN ATE % Error', 1): 0.04138872832019602,
  ('GCN ATE % Error', 2): 0.07992515441508585,
  ('GCN ATE % Error', 3): 0.010626119847234827,
  ('GCN ATE % Error', 4): 0.012687682422655833,
  ('GCN KL Divergence', 0): 0.30315243805992076,
  ('GCN KL Divergence', 1): 0.297137139109917

In [ ]:
gcn_error=[te_train[i] for i in te_train if 'GCN ATE % Error' in i[0]][3]
gcn_mse=[te_train[i] for i in te_train if 'GCN MSE' in i[0]][3]
gcn_kl=[te_train[i] for i in te_train if 'GCN KL Divergence' in i[0]][3]
gat_error=[te_train[i] for i in te_train if 'GAT ATE % Error' in i[0]][3]
gat_mse=[te_train[i] for i in te_train if 'GAT MSE' in i[0]][3]
gat_kl=[te_train[i] for i in te_train if 'GAT KL Divergence' in i[0]][3]
train_summary=pd.DataFrame(columns=[["Abs % Error of ATE","MSE","KL Divergence"]])
train_summary.loc["S learner(GCN)",:]=[gcn_error,gcn_mse,gcn_kl]
train_summary.loc["S learner(GAT)",:]=[gat_error,gat_mse,gat_kl]

gcn_error=[te_valid[i] for i in te_valid if 'GCN ATE % Error' in i[0]][3]
gcn_mse=[te_valid[i] for i in te_valid if 'GCN MSE' in i[0]][3]
gcn_kl=[te_valid[i] for i in te_valid if 'GCN KL Divergence' in i[0]][3]
gat_error=[te_valid[i] for i in te_valid if 'GAT ATE % Error' in i[0]][3]
gat_mse=[te_valid[i] for i in te_valid if 'GAT MSE' in i[0]][3]
gat_kl=[te_valid[i] for i in te_valid if 'GAT KL Divergence' in i[0]][3]
valid_summary=pd.DataFrame(columns=[["Abs % Error of ATE","MSE","KL Divergence"]])
valid_summary.loc["S learner(GCN)",:]=[gcn_error,gcn_mse,gcn_kl]
valid_summary.loc["S learner(GAT)",:]=[gat_error,gat_mse,gat_kl]

print("Validation Summary of Embedding(10d) & Causal Weighting(6d)")
valid_summary

Validation Summary of Embedding(10d) & Causal Weighting(6d)


,Abs % Error of ATE,MSE,KL Divergence
S learner(GCN),0.012653,0.128446,0.364849
S learner(GAT),0.4912,0.118681,0.629808


In [ ]:
train_summary.to_csv("Train_embedding(10d)_ate(6d).csv")
valid_summary.to_csv("Valid_embedding(10d)_ate(6d).csv")